In [14]:
import os 

In [15]:
%pwd

'c:\\Users\\Ghofrane\\MLOPsProject_ge\\MLOPsProject\\research'

In [16]:
os.chdir("../")


In [17]:
%pwd

'c:\\Users\\Ghofrane\\MLOPsProject_ge\\MLOPsProject'

In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [21]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-11 21:01:07,928: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-11 21:01:07,985: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-11 21:01:07,995: INFO: common: created directory at: artifacts]
[2025-12-11 21:01:08,002: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-11 21:01:08,007: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1z_kF708QABOdieYJ69_fQpeYxRMFajiy/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z_kF708QABOdieYJ69_fQpeYxRMFajiy
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z_kF708QABOdieYJ69_fQpeYxRMFajiy&confirm=t&uuid=b419af5b-4567-4807-83b0-4fb591089c31
To: c:\Users\Ghofrane\MLOPsProject_ge\MLOPsProject\artifacts\data_ingestion\data.zip
100%|██████████| 940M/940M [38:51<00:00, 403kB/s]   

[2025-12-11 21:40:04,449: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1z_kF708QABOdieYJ69_fQpeYxRMFajiy/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
